In [ ]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.train3d import *

In [ ]:
torch.cuda.set_device(2)

In [ ]:
dls_feat = get_3d_dls_feat(Meta.df_comb, path=path_feat_384avg, bs=32)

## Model

In [ ]:
xb, yb = dls_feat.one_batch(); xb.shape

In [ ]:
from torch.nn import TransformerEncoder, TransformerEncoderLayer
class SeqHead(nn.Module):
    def __init__(self):
        super().__init__()
#         d_model = 2048+6+1
        d_model = 1024
        n_head = 4
        
        self.flat = nn.Sequential(AdaptiveConcatPool2d(), Flatten())
        self.hook = ReshapeBodyHook(self.flat)
        
#         self.linear = nn.Linear(d_model+7, d_model)
        encoder_layers = TransformerEncoderLayer(d_model, n_head, d_model*2)
        self.transformer = TransformerEncoder(encoder_layers, 4)
        
        self.head = nn.Sequential(nn.Linear(d_model,6))
        
    def forward(self, x):
        x = self.flat(x)
#         x = torch.cat(x, axis=-1)
#         x = self.linear(x)
        feat = self.transformer(x.transpose(0,1))
        return self.head(feat.transpose(0,1))

In [ ]:
m = SeqHead()
name = 'train3d_baseline_feat_transformer'
learn = get_learner(dls_feat, m, name=name)
learn.add_cb(DePadLoss())

In [20]:
xb.shape

torch.Size([32, 60, 512, 8, 8])

In [ ]:
# with torch.no_grad():
#     learn.model(xb).shape

In [10]:
# learn.summary()

## Training

In [22]:
learn.lr_find()

torch.Size([32, 60, 1024])
torch.Size([32, 60, 1024])
torch.Size([32, 60, 1024])
torch.Size([32, 60, 1024])
torch.Size([32, 60, 1024])


KeyboardInterrupt: 

In [9]:
do_fit(learn, 10, 1e-3)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.054112,0.068720,0.977991,0.957986,01:25
1,0.053138,0.083151,0.972912,0.950538,01:24
2,0.051664,0.070177,0.977155,0.959221,01:20
3,0.048969,0.070529,0.977832,0.958712,01:19
4,0.052044,0.070904,0.977355,0.955452,01:20
5,0.049691,0.072793,0.977688,0.957642,01:20
6,0.049509,0.066312,0.978764,0.959246,01:20
7,0.048504,0.065887,0.979067,0.959616,01:20
8,0.048873,0.066936,0.979014,0.959666,01:31
9,0.046559,0.066987,0.978982,0.959386,01:24


## Testing

In [10]:
learn.dls = get_3d_dls_feat(Meta.df_tst, path=path_feat_tst_384avg, bs=32, test=True)

In [11]:
sub_fn = f'subm/{name}'
learn.load(f'runs/{name}-1')

In [12]:
preds,targs = learn.get_preds()
preds.shape, preds.min(), preds.max()

(torch.Size([121232, 6]), tensor(3.8507e-09), tensor(1.0000))

In [13]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [14]:
api.competition_submit(f'{sub_fn}.csv', name, 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:32<00:00, 846kB/s] 


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [19]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14320325,
 'totalBytes': 27277209,
 'date': '2020-01-29T08:20:57.827Z',
 'description': 'train3d_baseline_feat_lstm_2ndplace',
 'errorDescription': None,
 'fileName': 'train3d_baseline_feat_lstm_2ndplace.csv',
 'publicScore': '1.16287',
 'privateScore': '0.05369',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14320325/14320325.raw'}